In [ ]:
from pathlib import Path

import torch
from torch.utils.data import DataLoader

from src.data.dataset import ObjectDetectionDataset
from src.data.entry import (
    read_entries_from_directory,
    split_entries_train_val_test,
    equalize_negative_samples_with_positives
)
from src.data.transforms import get_inference_transform, get_training_transform
from src.data.visualize import plot_entries_original_and_annotated
from src.trainer import Trainer
from src.utils import load_config, seed_everything

print("Pytorch version:", torch.__version__)
print("CUDA enabled:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name())

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available else "cpu")
DATASET_DIR = Path("C:/Users/robert/Desktop/sem1/NN/datasets/DetectionPatches_512x512_ALL_No-Overlap")
CHECKPOINT_DIR = Path("models")
CONFIG_PATH = Path("config.json")
config = load_config(CONFIG_PATH)
seed_everything(seed=config["seed"])
config

In [ ]:
entries = read_entries_from_directory(
    data_dir=DATASET_DIR,
    leave_out_negatives=True,
    patch_size=config["image_size"],
)
entries = equalize_negative_samples_with_positives(entries)
train_entries, val_entries, test_entries = split_entries_train_val_test(entries)

In [ ]:
train_transform = get_training_transform(config["image_size"], config["in_channels"])
test_transform = get_inference_transform(config["image_size"], config["in_channels"])

full_dataset = ObjectDetectionDataset(entries, config=config, transforms=test_transform)
train_dataset = ObjectDetectionDataset(train_entries, config=config, transforms=train_transform)
val_dataset = ObjectDetectionDataset(val_entries, config=config, transforms=test_transform)
test_dataset = ObjectDetectionDataset(test_entries, config=config, transforms=test_transform)

print(full_dataset.describe("FULL"))
print(train_dataset.describe("TRAIN"))
print(val_dataset.describe("VAL"))
print(test_dataset.describe("TEST"))

In [ ]:
max_annotation_entry = max(entries, key=lambda x: len(x.annotations), default=None)

if max_annotation_entry is not None:
    plot_entries_original_and_annotated([max_annotation_entry], samples_to_display=1, linewidth=1)

In [ ]:
batch_size = config["batch_size"]
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [ ]:
trainer = Trainer(
    config=config,
    dataloaders=(train_dataloader, val_dataloader, test_dataloader),
    device=DEVICE,
    save_dir=CHECKPOINT_DIR,
    checkpoint_path=None
)

In [ ]:
trainer.fit()

In [ ]:
trainer.evaluate(test_dataloader, iou_threshold=0.5, threshold=0.4)